In [33]:
import pandas as pd
from category_encoders import TargetEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error

# Load data
df_train = pd.read_csv('../datasets/mod_04_hw_train_data.csv')
df_train

,Name,Phone_Number,Experience,Qualification,University,Role,Cert,Date_Of_Birth,Salary
0,Jennifer Hernandez,120-602-1220,3.0,Msc,Tier2,Mid,Yes,25/08/1972,98000
1,Timothy Walker,840-675-8650,5.0,PhD,Tier2,Senior,Yes,03/12/2013,135500
2,David Duran,556-293-8643,5.0,Msc,Tier2,Senior,Yes,19/07/2002,123500
3,Gloria Ortega,463-559-7474,3.0,Bsc,Tier3,Mid,No,19/02/1970,85000
4,Matthew Steele,968-091-7683,5.0,Bsc,Tier2,Senior,Yes,20/02/1970,111500
...,...,...,...,...,...,...,...,...,...
244,Adam Graham,107-988-5466,3.0,PhD,Tier2,Mid,No,12/01/1993,103000
245,Christopher Brown,376-543-7355,3.0,Msc,Tier1,Junior,Yes,26/08/2001,76500
246,Emily Brown,553-413-5579,5.0,Bsc,Tier1,Senior,Yes,08/02/1980,107500
247,Mrs. Audrey Ware,578-962-7818,5.0,PhD,Tier3,Senior,No,11/10/1998,134500


### Розглянувши основні статистичні показники цього датасету можна прийти до наступних висновків

1. Значна варіативність наступних колонок: Name, Phone_Number, Date_Of_Birth
2. Незначна кількість пропущених даних у колонках: Role, Experience, Cert, Qualification

* Для підготовки датасету до подальшого аналізу ми видалимо малоінформативні колонки та записи з пропущеними значеннями.

In [34]:
df_train.drop(columns=['Name', 'Phone_Number', 'Date_Of_Birth'], inplace=True)
df_train.dropna(inplace=True)

### Наступні кроки

* Роділимо дані на тренувальну і тестову вибірки
* Закодуємо категоріальні фічі за допомогою `Target Encoder`
* Приведемо дані до єдиного масштабу
* Натренуємо модель та виведемо метрики

In [39]:
categorical_columns = ['Qualification', 'University', 'Role', 'Cert']

# Define X and y
X = df_train.drop(columns=['Salary'])
y = df_train['Salary']

# Розібємо датасет на тренувальну та тестову вибірки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Закодуємо категоріальні фічі
encoder = TargetEncoder(cols=categorical_columns)
encoder.set_output(transform='pandas')
X_train[categorical_columns] = encoder.fit_transform(X_train[categorical_columns], y_train)
X_test[categorical_columns] = encoder.transform(X_test[categorical_columns])

# Приведемо їх до єдиного масштабу
st_scaler = StandardScaler()
X_train = st_scaler.fit_transform(X_train)
X_test = st_scaler.transform(X_test)

# Тренуємо модель
model = KNeighborsRegressor(n_neighbors=5)
model.fit(X_train, y_train)

# Робимо заміри продуктивності моделі і виводимо метрики
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)

print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"R² Score: {r2:.4f}")
print(f"Mean Absolute Percentage Error (MAPE): {mape:.4%}")

Mean Squared Error (MSE): 25162653.0612
R² Score: 0.9514
Mean Absolute Percentage Error (MAPE): 3.8659%


### Висновок по метрикам:
* MSE (20699795.92) – середньоквадратична помилка вказує на значні відхилення у квадраті, але для моделі KNN це очікувано через масштаб зарплат.
* R² Score (0.96) – дуже високе значення, що свідчить про те, що модель добре пояснює варіацію зарплат за допомогою вхідних фіч.
* MAPE (3.54%) – низьке середнє абсолютне відхилення у відсотках, що свідчить про високу точність передбачень відносно реальних зарплат.

✅ Модель добре працює, точність передбачень висока, але варто перевірити узагальнюваність на валідаційних даних. 🚀

In [40]:
df_valid = pd.read_csv('../datasets/mod_04_hw_valid_data.csv')

# Видаляємо малоінформативні фічі та рядки з пропущеними значеннями
df_valid.drop(columns=['Name', 'Phone_Number', 'Date_Of_Birth'], inplace=True)
df_valid.dropna(inplace=True)

# Розібємо датасет на тренувальну та тестову вибірки
X_valid = df_valid.drop(columns=['Salary'])
y_valid = df_valid['Salary']

# Закодуємо категоріальні фічі
X_valid_encoded = encoder.transform(X_valid[categorical_columns])
X_valid[categorical_columns] = X_valid_encoded

# Приведемо їх до єдиного масштабу
X_valid = st_scaler.transform(X_valid)

# Робимо заміри продуктивності моделі і виводимо метрики
y_valid_pred = model.predict(X_valid)
mse_valid = mean_squared_error(y_valid, y_valid_pred)
r2_valid = r2_score(y_valid, y_valid_pred)
mape_valid = mean_absolute_percentage_error(y_valid, y_valid_pred)
print(f"Validation Mean Squared Error (MSE): {mse_valid:.4f}")
print(f"Validation R² Score: {r2_valid:.4f}")
print(f"Validation Mean Absolute Percentage Error (MAPE): {mape_valid:.4%}")

Validation Mean Squared Error (MSE): 116648571.4286
Validation R² Score: 0.5393
Validation Mean Absolute Percentage Error (MAPE): 10.7937%


### Висновок по валідаційних метриках:
* MSE – значно більше, ніж на тестових даних, що вказує на велику помилку передбачень та можливий проблеми з узагальненням.
* R² Score (0.53) – різке падіння порівняно з тренувальними даними (було 0.95), що означає, що модель погано пояснює варіацію зарплат у валідаційному наборі.
* MAPE (10.7937%) – значно вище, ніж на тестових даних (було 3.86%), що вказує на високий рівень помилки у відсотках.
* Спроби додати PowerTransformer до Salary не дали результату і навіть дещо погіршили метрики валідаційних даних
* Також спроби підібрати кращі параметри за допомогою GridSearch не дали приросту продуктивності
